In [ ]:
from matplotlib import pyplot as plt
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import math
from sklearn.model_selection import StratifiedKFold

## Check if there Null data

In [ ]:
for i in range(1, 106):
    df = pd.read_csv('ds'+ str(i) +'.csv')
    print('='*50)
    print('shape of {}:'.format(i), df.shape)
    print(df.columns)                            # features names and label
    if df.isnull().sum().sum() != 0:             # null?
        print(df.isnull().sum())
        break

## Experiment setting

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from sklearn import metrics
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from matplotlib import pyplot as plt
from keras.layers import BatchNormalization, Dropout
from keras.layers import Activation
from keras import optimizers
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import EarlyStopping

In [ ]:
epochs = 50
batch_size = 32
hidden_node = 2
learning_rate=0.001
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 0)

def mlp_model():
    model = Sequential()
    model.add(Dense(hidden_node, input_dim=X.shape[1], kernel_initializer=keras.initializers.he_normal(seed=100)))
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    return model

# 1. Generating Probablistic Labels for 102 datasets

In [ ]:
# Experiments for 105-3 Datasets

for i in range(1, 106):
    if i == 23 or i == 82 or i == 84:
        continue
    df = pd.read_csv('ds'+ str(i) +'.csv')
    print('+'*35, '{}th Dataset'.format(i), '+'*35)
    print('<Original Class>\n', df.iloc[:,-1].value_counts())
    
    # Make major class as '0' and minor class as '1'
    MAJOR = df.iloc[:,-1].value_counts()[df.iloc[:,-1].value_counts() == max(df.iloc[:,-1].value_counts())].index[0]
    minor = df.iloc[:,-1].value_counts()[df.iloc[:,-1].value_counts() != max(df.iloc[:,-1].value_counts())].index[0]
    df.iloc[:,-1] = df.iloc[:,-1].replace(MAJOR, -100)
    df.iloc[:,-1] = df.iloc[:,-1].replace(minor, 1)
    df.iloc[:,-1] = df.iloc[:,-1].replace(-100, 0)
    print('<Modified Class>\n', df.iloc[:,-1].value_counts())
    print('<Imabalance ratio>\n', "{: .2f}:1".format(df.iloc[:,-1].value_counts()[0]/df.iloc[:,-1].value_counts()[1]))
    
    X = df.iloc[:, :-1]
    X = (X - X.mean())/X.std()    # Features // Standardization
    y = df.iloc[:, -1]

    model = mlp_model()
    opt = optimizers.Adam(learning_rate = learning_rate)
    batch_size = int(X.shape[0] * 0.05)
    model.compile(loss='BinaryCrossentropy', optimizer=opt, metrics=['accuracy'])
    history = model.fit(X, y, validation_split=0.2, epochs=epochs, verbose=0, batch_size=batch_size)   
#     plt.plot(history.history['loss'], label='loss')
#     plt.ylim([0, 1])
#     plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#     plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#     plt.title("Cost Function",fontweight="bold",fontsize = 20)
#     plt.legend()
#     plt.show()
    result = model.predict(X)
    prob_label = list(result.reshape(len(X),))
    df['prob_label'] = prob_label
    df.to_csv("Prob_102datasets/ds{}_prob.csv".format(i), mode = 'a', float_format='%.4g')

# 1-1. Focal(Hard) and SLS(Hard/diverse alphas)_option#1

In [ ]:
for i in range(1, 106):
    if i == 23 or i == 82 or i == 84:
        continue
    df = pd.read_csv('ds'+str(i)+'_prob'+'.csv', index_col='Unnamed: 0')
    y_hard = df.iloc[:, -2]
    print('+'*35, '{}th Dataset'.format(i), '+'*35)
    print('<Original Class>\n', y_hard.value_counts())
    
    # Make major class as '0' and minor class as '1'
    MAJOR = y_hard.value_counts()[y_hard.value_counts() == max(y_hard.value_counts())].index[0]
    minor = y_hard.value_counts()[y_hard.value_counts() != max(y_hard.value_counts())].index[0]
    y_hard = y_hard.replace(MAJOR, -100)
    y_hard = y_hard.replace(minor, 1)
    y_hard = y_hard.replace(-100, 0)
    print('<Modified Class>\n', y_hard.value_counts())
    print('<Imabalance ratio>\n', "{: .2f}:1".format(y_hard.value_counts()[0]/y_hard.value_counts()[1]))
    
    X = df.iloc[:, :-2]
    X = (X - X.mean())/X.std()    # Features // Standardization
    y = df.iloc[:, -1]      # prob_label
    
    res = pd.DataFrame({'Dataset':[0, 0, 0, 0, 0]}, index = ['Acc','Pre','Rec','F1','R-AUC'])
    res.iloc[:,0] = [i for b in range(5)]
    
    # Focal
    print('#'*50,'Focal','#'*50)
    list_acc = []
    list_pre = []
    list_rec = []
    list_f1 = []
    list_rauc = []   
    focal_model = mlp_model()

    n_iter = 0
    for train_index, test_index in skf.split(X, y_hard):  # straticiation by y_hard(binary label)
        n_iter += 1
        X_train = X.iloc[train_index]
        y_train= y_hard.iloc[train_index]
        if n_iter == 1:
            print(y_train.value_counts())
        X_test = X.iloc[test_index]
        y_test= y_hard.iloc[test_index]
#         print('#'*10,'{0}th CV'.format(n_iter),'#'*10)
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        y_train = y_train.astype(float)
        X_test = np.array(X_test)
        y_test = np.array(y_test)
        y_test = y_test.astype(float)

        focal_model.compile(loss='BinaryFocalCrossentropy', optimizer=optimizers.Adam(learning_rate = 0.001), metrics=['accuracy'])
        history = focal_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch_size)#, callbacks=[early_stopping])
#         plt.plot(history.history['loss'], label='loss')
#         plt.ylim([0, 1])
#         plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#         plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#         plt.title("Cost Function",fontweight="bold",fontsize = 20)
#         plt.legend()
#         plt.show()
        predicted = np.round(focal_model.predict(X_test, verbose=0))
        list_acc.append(metrics.accuracy_score(y_test, predicted))
        list_pre.append(metrics.precision_score(y_test, predicted))
        list_rec.append(metrics.recall_score(y_test, predicted))
        list_f1.append(metrics.f1_score(y_test, predicted))
        list_rauc.append(metrics.roc_auc_score(y_test, predicted))
    res['Focal'] = [np.mean(list_acc), np.mean(list_pre), np.mean(list_rec), np.mean(list_f1), np.mean(list_rauc)]
    print([np.mean(list_acc), np.mean(list_pre), np.mean(list_rec), np.mean(list_f1), np.mean(list_rauc)])
    
    B = [0.00, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10]     # SLS with LS/NLS
    c = 0.3  # criterion decides easy/hard
    for b in B:
        print('#'*50,'SLS',b,'#'*50)
        y_005 = []
        for i in range(len(y_hard)):
            if y_hard[i] == 0:
                if y[i] <= c:
                    y_005.append(b)  # easy sample
                else:
                    y_005.append(0) # (or 0-b) hard sample
            if y_hard[i] == 1:
                if y[i] >= 1-c:
                    y_005.append(1-b)  # easy sample
                else:
                    y_005.append(1) # (or 1+b) hard sample
        y_005 = pd.DataFrame(y_005)      

        bce005_acc = []
        bce005_pre = []
        bce005_rec = []
        bce005_f1 = []
        bce005_rocauc = []
        model_005 = mlp_model()
#             early_stopping = EarlyStopping(monitor='val_loss', patience=5)
        n_iter = 0
        for train_index, test_index in skf.split(X, y_hard):  # straticiation by y_hard(binary label)
            n_iter += 1
            X_train = X.iloc[train_index]
            y_005_train= y_005.iloc[train_index]
            if n_iter == 1:
                print(y_005_train.value_counts())
            X_test = X.iloc[test_index]
            y_test= y_hard.iloc[test_index]  # test with real(actual) label y
#                 print('#'*10,'{0}th CV'.format(n_iter),'#'*10)
            X_train = np.array(X_train)
            y_005_train = np.array(y_005_train)
            y_005_train = y_005_train.astype(float)
            X_test = np.array(X_test)
            y_test = np.array(y_test)
            y_test = y_test.astype(float)

            # MLP_BCE(y_005)
            model_005.compile(loss='BinaryCrossentropy', optimizer=optimizers.Adam(learning_rate = 0.001), metrics=['accuracy'])
            history = model_005.fit(X_train, y_005_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch_size)#, callbacks=[early_stopping])
#             plt.plot(history.history['loss'], label='loss')
#             plt.ylim([0, 1])
#             plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#             plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#             plt.title("Cost Function",fontweight="bold",fontsize = 20)
#             plt.legend()
#             plt.show()
            predicted = np.round(model_005.predict(X_test, verbose=0))
            bce005_acc.append(metrics.accuracy_score(y_test, predicted))
            bce005_pre.append(metrics.precision_score(y_test, predicted))
            bce005_rec.append(metrics.recall_score(y_test, predicted))
            bce005_f1.append(metrics.f1_score(y_test, predicted))
            bce005_rocauc.append(metrics.roc_auc_score(y_test, predicted))
        res['SLS({})'.format(b)] = [np.mean(bce005_acc), np.mean(bce005_pre), np.mean(bce005_rec), np.mean(bce005_f1), np.mean(bce005_rocauc)]
        print([np.mean(bce005_acc), np.mean(bce005_pre), np.mean(bce005_rec), np.mean(bce005_f1), np.mean(bce005_rocauc)])
    res.to_csv("102datasets_5CV(SLS_opt#1_c0.3).csv", mode = 'a', float_format='%.4g')